In [9]:
using JLD2
using Statistics
using DataFrames

In [10]:
algs = ["central"; "central"; "upwind"; "upwind"; "muscl"; "muscl"; "muscl"; "muscl"];
orders = [1; 2; 1; 2; 1; 2; 3; 4];
totalNumGrids = 100;

In [11]:
dataFolderMaxNoise = "$(@__DIR__)/data1DMaxNoise/"  # Noise: dx/2
dataFolderNoise1 = "$(@__DIR__)/data1DNoise1/"  # Noise: dx/2 - 0.05*dx
dataFolderNoise2 = "$(@__DIR__)/data1DNoise2/"  # Noise: dx/2 - 0.1*dx

function loadData(dataFolder)
    m1 = Matrix{Float64}(undef, convert(Int64, length(readdir(dataFolder))/2), length(orders))
    N1 = Vector{Int64}(undef, convert(Int64, length(readdir(dataFolder))/2))
    m2 = Matrix{Float64}(undef, convert(Int64, length(readdir(dataFolder))/2), length(orders))
    N2 = Vector{Int64}(undef, convert(Int64, length(readdir(dataFolder))/2))

    i1 = 1
    i2 = 1
    for file in readdir(dataFolder)
        d = load(dataFolder*file)
        weight = split(file, "_")[end][1:end-5]
        unstableSim = map(x -> x > 0.0 ? true : false, d["res"])
        unstableSims = mapslices(count, unstableSim, dims=1)
        if occursin("exp", file)
            m1[i1, :] .= 100*unstableSims[:]/totalNumGrids
            N1[i1] = d["N"]
            i1 += 1
        else
            m2[i2, :] .= 100*unstableSims[:]/totalNumGrids
            N2[i2] = d["N"]
            i2 += 1
        end
    end

    dfExp = DataFrame(m1, ["$(alg) $(order)" for (alg, order) in zip(algs, orders)]);
    dfExp[!, "N"] = N1;
    sort!(dfExp, :N);
    dfinverse = DataFrame(m2, ["$(alg) $(order)" for (alg, order) in zip(algs, orders)]);
    dfinverse[!, "N"] = N2;
    sort!(dfinverse, :N);
    return dfExp, dfinverse
end

dfExpMax, dfInverseMax = loadData(dataFolderMaxNoise);
dfExpNoise1, dfInverseNoise1 = loadData(dataFolderNoise1);
dfExpNoise2, dfInverseNoise2 = loadData(dataFolderNoise2);

In [12]:
println(dfExpMax)
println(dfInverseMax)

4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      2.0    100
   2 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      5.0    200
   3 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      5.0    300
   4 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      5.0    400
4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0     66.0

In [13]:
println(dfExpNoise1)
println(dfInverseNoise1)

4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      0.0    100
   2 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      1.0    200
   3 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      2.0    300
   4 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      3.0    400
4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0     42.0

In [14]:
println(dfExpNoise2)
println(dfInverseNoise2)

4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0     97.0      0.0    100.0      0.0    100
   2 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      0.0    200
   3 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      0.0    300
   4 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      0.0    400
4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0     15.0

In [15]:
file = load("$(@__DIR__)/data1DMaxNoise/gridSensitivity1D_300_expWeight.jld2")
eigs = file["eigs"]

for i = 1:length(algs)
    println("Alg: $(algs[i]), order: $(orders[i]), unstable eigenvalues: $([x for x in eigs[:, i] if x != 0.0])")
end

Alg: central, order: 1, unstable eigenvalues: ComplexF64[7.625121196033515 + 0.0im, 12.415443484939464 + 0.0im, 3.6501516117456374 + 0.0im, 5.782039071066811 + 0.0im, 5.804578679850332 + 0.0im, 18.265302525010192 + 0.0im, 5.887416281159298 + 0.0im, 10.366131673783876 + 0.0im, 3.9175585610018975 + 0.0im, 5.781009794698859 + 0.0im, 5.331786171933761 + 0.0im, 3.3380692949191513 + 0.0im, 9.270411510647554 + 0.0im, 6.792527263445177 + 0.0im, 11.108931128228058 + 0.0im, 5.923087834460697 + 0.0im, 5.98822701595409 + 0.0im, 3.9402522351037446 + 0.0im, 15.848314872260007 + 0.0im, 14.309545330095478 + 0.0im, 10.517456830860104 + 0.0im, 3.877857022517623 + 0.0im, 3.973039978748626 + 0.0im, 5.487174909087977 + 0.0im, 6.18528072860662 + 0.0im, 5.977017812554515 + 0.0im, 7.349272725946813 + 0.0im, 6.239376753066317 + 0.0im, 11.675416511556023 + 0.0im, 11.836361818106845 + 0.0im, 10.897373693707092 + 0.0im, 12.30001962133829 + 0.0im, 4.00133016183481 + 0.0im, 7.9785210778630695 + 0.0im, 17.0755081963